In [1]:
import time 
from bs4 import BeautifulSoup
import requests
import pandas as pd
import random
import datetime
import csv


In [6]:
from playwright.async_api import async_playwright


In [7]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)


In [8]:
browser.new_page()

<coroutine object Browser.new_page at 0x12d104790>

In [9]:
browser.new_page()
page = await browser.new_page()


/var/folders/j_/_3yzbdjs3ylf4k27zrqrssqc0000gp/T/ipykernel_95274/3314845844.py:1: RuntimeWarning: coroutine 'Browser.new_page' was never awaited
  browser.new_page()


In [10]:
#new jupyter notebook to scrape the list of date and the corresponding website link

my_url_sample = 'http://sousuo.gov.cn/column/40403/0.htm'
requests = requests.get(my_url_sample)
soup = BeautifulSoup(requests.text, 'html.parser')

In [11]:
covid_daily_announcement = soup.find_all('ul', class_ = 'listTxt')
covid_daily_announcement = covid_daily_announcement[0].find_all('a')
covid_daily_announcement
# len(covid_daily_announcement)

# daily_announce_links = [i['href'] for i in covid_daily_announcement[0].find_all('a')]
# daily_announce_titles = [i.text for i in covid_daily_announcement[0].find_all('a')]



[<a href="http://www.gov.cn/xinwen/2022-12/13/content_5731660.htm" target="_blank">截至12月12日24时新型冠状病毒肺炎疫情最新情况</a>,
 <a href="http://www.gov.cn/xinwen/2022-12/12/content_5731552.htm" target="_blank">新冠病毒疫苗接种情况(截至12月11日)</a>,
 <a href="http://www.gov.cn/xinwen/2022-12/12/content_5731442.htm" target="_blank">截至12月11日24时新型冠状病毒肺炎疫情最新情况</a>,
 <a href="http://www.gov.cn/xinwen/2022-12/11/content_5731399.htm" target="_blank">助企纾困加力 多地落实落细系列税费支持政策</a>,
 <a href="http://www.gov.cn/xinwen/2022-12/11/content_5731373.htm" target="_blank">新冠病毒疫苗接种情况(截至12月10日)</a>,
 <a href="http://www.gov.cn/fuwu/2022-12/11/content_5731356.htm" target="_blank">2023"我向总理说句话"网民建言征集活动开始、乘火车不再查验48小时核酸阴性证明……本周提醒来了!</a>,
 <a href="http://www.gov.cn/xinwen/2022-12/11/content_5731342.htm" target="_blank">截至12月10日24时新型冠状病毒肺炎疫情最新情况</a>,
 <a href="http://www.gov.cn/xinwen/2022-12/11/content_5731333.htm" target="_blank">财政部:2022年特别国债为到期特别国债的等额滚动发行</a>,
 <a href="http://www.gov.cn/xinwen/2022-12/11/content_5731324.htm" target="_b

In [12]:
# for single page:

links_per_page = []
for i in range(0, len(covid_daily_announcement)):
    daily_pair  = {}
    daily_pair['date'] = ''
    daily_pair['title'] = covid_daily_announcement[i].text
    daily_pair['link'] = covid_daily_announcement[i]['href']
    if '新冠病毒疫苗接种情况' in daily_pair['title']:
        daily_pair['date'] = daily_pair['link'][25:35]
        daily_pair['date'] = datetime.datetime.strptime(daily_pair['date'], '%Y-%m/%d').date()
        print(daily_pair)
        links_per_page.append(daily_pair)
links_per_page



{'date': datetime.date(2022, 12, 12), 'title': '新冠病毒疫苗接种情况(截至12月11日)', 'link': 'http://www.gov.cn/xinwen/2022-12/12/content_5731552.htm'}
{'date': datetime.date(2022, 12, 11), 'title': '新冠病毒疫苗接种情况(截至12月10日)', 'link': 'http://www.gov.cn/xinwen/2022-12/11/content_5731373.htm'}
{'date': datetime.date(2022, 12, 10), 'title': '新冠病毒疫苗接种情况(截至12月9日)', 'link': 'http://www.gov.cn/xinwen/2022-12/10/content_5731258.htm'}


[{'date': datetime.date(2022, 12, 12),
  'title': '新冠病毒疫苗接种情况(截至12月11日)',
  'link': 'http://www.gov.cn/xinwen/2022-12/12/content_5731552.htm'},
 {'date': datetime.date(2022, 12, 11),
  'title': '新冠病毒疫苗接种情况(截至12月10日)',
  'link': 'http://www.gov.cn/xinwen/2022-12/11/content_5731373.htm'},
 {'date': datetime.date(2022, 12, 10),
  'title': '新冠病毒疫苗接种情况(截至12月9日)',
  'link': 'http://www.gov.cn/xinwen/2022-12/10/content_5731258.htm'}]

In [13]:
# to csv
with open('daily_links.csv', 'w', newline='') as csvfile:
    fieldnames = ['date', 'title', 'link']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for i in range(0, len(links_per_page)):
        writer.writerow(links_per_page[i])


In [14]:
# for multiple pages:
import requests
import pandas as pd
import random
import datetime
import csv
from bs4 import BeautifulSoup
import time

all_links = []
page_number = list(range(0,1))

for page in page_number:
    my_url = 'http://sousuo.gov.cn/column/40403/' + str(page) + '.htm'
    response = requests.get(my_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    covid_daily_announcement = soup.find_all('ul', class_ = 'listTxt')
    covid_daily_announcement = covid_daily_announcement[0].find_all('a')
    links_per_page = []
    for i in range(0, len(covid_daily_announcement)):
        daily_pair  = {}
        daily_pair['date'] = ''
        daily_pair['title'] = covid_daily_announcement[i].text
        daily_pair['link'] = covid_daily_announcement[i]['href']
        if '新冠病毒疫苗接种情况' in daily_pair['title']:
            daily_pair['date'] = daily_pair['link'][25:35]
            daily_pair['date'] = datetime.datetime.strptime(daily_pair['date'], '%Y-%m/%d').date()
            print(daily_pair)
            links_per_page.append(daily_pair)
            # write to csv
            with open('links.csv', 'a', newline='') as csvfile:
                fieldnames = ['date', 'title', 'link']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writerow(daily_pair)
    all_links.append(links_per_page)
    time.sleep(random.randint(1,3))

all_links

{'date': datetime.date(2022, 12, 12), 'title': '新冠病毒疫苗接种情况(截至12月11日)', 'link': 'http://www.gov.cn/xinwen/2022-12/12/content_5731552.htm'}
{'date': datetime.date(2022, 12, 11), 'title': '新冠病毒疫苗接种情况(截至12月10日)', 'link': 'http://www.gov.cn/xinwen/2022-12/11/content_5731373.htm'}
{'date': datetime.date(2022, 12, 10), 'title': '新冠病毒疫苗接种情况(截至12月9日)', 'link': 'http://www.gov.cn/xinwen/2022-12/10/content_5731258.htm'}


[[{'date': datetime.date(2022, 12, 12),
   'title': '新冠病毒疫苗接种情况(截至12月11日)',
   'link': 'http://www.gov.cn/xinwen/2022-12/12/content_5731552.htm'},
  {'date': datetime.date(2022, 12, 11),
   'title': '新冠病毒疫苗接种情况(截至12月10日)',
   'link': 'http://www.gov.cn/xinwen/2022-12/11/content_5731373.htm'},
  {'date': datetime.date(2022, 12, 10),
   'title': '新冠病毒疫苗接种情况(截至12月9日)',
   'link': 'http://www.gov.cn/xinwen/2022-12/10/content_5731258.htm'}]]

In [15]:
df = pd.read_csv('links.csv', index_col = 0, names=['date', 'title', 'link'])

In [16]:
df.link[0]

'http://www.gov.cn/xinwen/2022-12/12/content_5731552.htm'

In [17]:
# from playwright.async_api import async_playwright

# playwright = await async_playwright().start()
# browser = await playwright.chromium.launch(headless = False)
# browser.new_page()
# page = await browser.new_page()

In [127]:
import requests

cookies = {
    'sVoELocvxVW0S': '5SCfB82xekZkf1z8Llz4svA6L2nTZOG34XN5ZPdjmB3YqtlpAs25wB.4N16YWlv697kIeWi_XBteAfB4CfWfLka',
    'yfx_c_g_u_id_10006654': '_ck22112421261414475334743259433',
    'wzws_sessionid': 'gTg2ZWE4MqBjiqIegjkwZGE0YoAxNTguMjIyLjE5OC4xMTk=',
    'insert_cookie': '96260894',
    'arialoadData': 'true',
    'ariawapChangeViewPort': 'false',
    '_gscu_2059686908': '70906610vucb9d20',
    '_gscbrs_2059686908': '1',
    '_gscs_2059686908': 't7096304056jq0t21|pv:4',
    'yfx_f_l_v_t_10006654': 'f_t_1669343174444__r_t_1670955302867__v_t_1670964763818__r_c_5',
    'sVoELocvxVW0T': '53yOv1bi3y07qqqDmkVIH6abNj0r12rL3ihoINS0zjXgWcthFQYfMB..61r7_ZASkklECD1Seo7y0gCjlDzTBvJOTvXRLApPl4jUbPbM5J6724EpverDNgUPLTZjdABRL8chgsrSIAWNtFONAsEyuCyoEuq6prdTryeRl7cEDjhXANNfgTPMFbYi0drG5bE7hcojhdrKMIoSVi34u8kF7SgwmT0fVy8dYx1LRDul2KTpwkt_wixphix7psIkg35rzeTeOmtSFFSzBzYR.mHikd1wi8lEREfGXF5YuTXjnrZKkHDR3ivqhOAM6wc39Alu_.RSNMkLo5EaH0HBNqJ.IWa',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'en-US,en;q=0.9,zh-CN;q=0.8,zh-TW;q=0.7,zh;q=0.6',
    'Connection': 'keep-alive',
    # 'Cookie': 'sVoELocvxVW0S=5SCfB82xekZkf1z8Llz4svA6L2nTZOG34XN5ZPdjmB3YqtlpAs25wB.4N16YWlv697kIeWi_XBteAfB4CfWfLka; yfx_c_g_u_id_10006654=_ck22112421261414475334743259433; wzws_sessionid=gTg2ZWE4MqBjiqIegjkwZGE0YoAxNTguMjIyLjE5OC4xMTk=; insert_cookie=96260894; arialoadData=true; ariawapChangeViewPort=false; _gscu_2059686908=70906610vucb9d20; _gscbrs_2059686908=1; _gscs_2059686908=t7096304056jq0t21|pv:4; yfx_f_l_v_t_10006654=f_t_1669343174444__r_t_1670955302867__v_t_1670964763818__r_c_5; sVoELocvxVW0T=53yOv1bi3y07qqqDmkVIH6abNj0r12rL3ihoINS0zjXgWcthFQYfMB..61r7_ZASkklECD1Seo7y0gCjlDzTBvJOTvXRLApPl4jUbPbM5J6724EpverDNgUPLTZjdABRL8chgsrSIAWNtFONAsEyuCyoEuq6prdTryeRl7cEDjhXANNfgTPMFbYi0drG5bE7hcojhdrKMIoSVi34u8kF7SgwmT0fVy8dYx1LRDul2KTpwkt_wixphix7psIkg35rzeTeOmtSFFSzBzYR.mHikd1wi8lEREfGXF5YuTXjnrZKkHDR3ivqhOAM6wc39Alu_.RSNMkLo5EaH0HBNqJ.IWa',
    'DNT': '1',
    'Referer': 'http://www.nhc.gov.cn/xcs/yqjzqk/list_gzbd_26.shtml',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
}

response = requests.get('http://www.nhc.gov.cn/xcs/yqjzqk/list_gzbd_27.shtml', cookies=cookies, headers=headers, verify=False)

In [128]:
response.headers
soup = BeautifulSoup(response.text, 'html.parser')

ml = soup.find_all('li')

for link in ml:
    single_row  = {'date':'', 'title':'', 'link':''}
    single_row['date'] = pd.to_datetime(link.span.text).date()+datetime.timedelta(days=-1)
    single_row['title'] = link.a.text
    single_row['link'] = 'http://www.nhc.gov.cn'+link.a['href']
    print(single_row)
    #write to csv
    with open('data.csv', 'a', newline='') as csvfile:
        fieldnames = ['date', 'title', 'link']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writerow(single_row)



{'date': datetime.date(2021, 3, 28), 'title': '全国新冠病毒疫苗接种情况', 'link': 'http://www.nhc.gov.cn/xcs/yqjzqk/202103/06db88dae17445fb82a6ba832efe9cf7.shtml'}
{'date': datetime.date(2021, 3, 27), 'title': '全国新冠病毒疫苗接种情况', 'link': 'http://www.nhc.gov.cn/xcs/yqjzqk/202103/575af2e2a7794f0b9e761e2c0390b69b.shtml'}
{'date': datetime.date(2021, 3, 26), 'title': '全国新冠病毒疫苗接种情况', 'link': 'http://www.nhc.gov.cn/xcs/yqjzqk/202103/0e5bf66b833a45de8db07e706775ae8e.shtml'}
{'date': datetime.date(2021, 3, 25), 'title': '全国新冠病毒疫苗接种情况', 'link': 'http://www.nhc.gov.cn/jkj/s7915/202103/0d1b93ae500c418db1955fb6b4ffee70.shtml'}
{'date': datetime.date(2021, 3, 24), 'title': '全国新冠病毒疫苗接种情况', 'link': 'http://www.nhc.gov.cn/jkj/s7915/202103/1d85a9ad629a4abcb4165803eb546b7a.shtml'}
{'date': datetime.date(2021, 3, 23), 'title': '全国新冠病毒疫苗接种情况', 'link': 'http://www.nhc.gov.cn/jkj/s7915/202103/4e9f38edab0243e6ac433a181906e527.shtml'}


In [9]:
vaccine_link = pd.read_csv('data.csv')


In [10]:
vaccine_link.value_counts().sort_index

<bound method Series.sort_index of 2022-12-12  新冠病毒疫苗接种情况    http://www.nhc.gov.cn/xcs/yqjzqk/202212/e2b3e84dad9a4dd69e2b333af7e401c6.shtml
2021-03-23  全国新冠病毒疫苗接种情况  http://www.nhc.gov.cn/jkj/s7915/202103/4e9f38edab0243e6ac433a181906e527.shtml     1
2022-05-20  新冠病毒疫苗接种情况    http://www.nhc.gov.cn/jkj/s7915/202205/10195c42e4c149d5bdff70827bdd5f15.shtml     1
2022-05-13  新冠病毒疫苗接种情况    http://www.nhc.gov.cn/jkj/s7915/202205/576f121dc47f417f8ad3fc9c681b98be.shtml     1
2022-05-14  新冠病毒疫苗接种情况    http://www.nhc.gov.cn/jkj/s7915/202205/b7bee38d0b644b4dbe9e78f3725ddd0a.shtml     1
2022-05-15  新冠病毒疫苗接种情况    http://www.nhc.gov.cn/jkj/s7915/202205/daa4ab0890c547c8b35b99cc769b2523.shtml     1
                                                                                                           ..
2021-10-19  新冠病毒疫苗接种情况    http://www.nhc.gov.cn/jkj/s7915/202110/d43226f89b3f4977813f35bd6b9acd2b.shtml     1
2021-10-20  新冠病毒疫苗接种情况    http://www.nhc.gov.cn/jkj/s7915/202110/febf34f3e8ec435cbf8a0070b

In [39]:
url = 'http://www.nhc.gov.cn/jkj/s7915/202110/d43226f89b3f4977813f35bd6b9acd2b.shtml'

import asyncio
from pyppeteer import launch

import time 
from bs4 import BeautifulSoup
import requests
import pandas as pd
import random
import datetime
import csv

import os
import asyncio
from pyppeteer import launch
from bs4 import BeautifulSoup

import nest_asyncio
nest_asyncio.apply()


In [42]:

url = 'http://www.nhc.gov.cn/jkj/s7915/202110/d43226f89b3f4977813f35bd6b9acd2b.shtml'

async def main():
    browser = await launch({'headless': False})
    page = await browser.newPage()
    await page.goto(url)
    await asyncio.wait([page.waitForNavigation()])
    str = await page.content()
    soup = BeautifulSoup(str, 'html.parser')
    soup.find_all('p')
    for p in soup.find_all('p'):
        print(p.text)

asyncio.get_event_loop().run_until_complete(main())

TypeError: Passing coroutines is forbidden, use tasks explicitly.

In [20]:
def getContent(html):
        
    bsobj = BeautifulSoup(html,'html.parser')
    cnt = bsobj.find('div', attrs={"id":"xw_box"}).find_all("p")
    s = ""
    if cnt:
        for item in cnt:
            s += item.text
        return s

    return "爬取失败！"

